### Initializing CMBAgent

In [118]:
import os
import re
import numpy as np
import pandas as pd
import copy
import os
import glob
import json

os.environ["CMBAGENT_DEBUG"] = "false"
os.environ["ASTROPILOT_DISABLE_DISPLAY"] = "false"

import cmbagent
path_to_targets = '/Users/milind24/cmbagentmain/cmbagent'

### Making and Testing the samples to run

In [119]:
# Install needed packages if not already installed
# !pip install datasets

from datasets import load_dataset
import traceback

# 1. Load top 10 samples from DS-1000
ds = load_dataset("xlangai/DS-1000", split="test")
#samples = ds.shuffle(seed=42).select(range(1))  # Top 10 random samples
#samples = ds.select(range(1))
samples = ds.select([0])

### The prompt which goes into CMBAgent along with other instructiions

In [120]:
print(f"{samples['prompt'][0]}")

Problem:
I have the following DataFrame:
    Col1  Col2  Col3  Type
0      1     2     3     1
1      4     5     6     1
2      7     8     9     2
3    10    11    12     2
4    13    14    15     3
5    16    17    18     3


The DataFrame is read from a CSV file. All rows which have Type 1 are on top, followed by the rows with Type 2, followed by the rows with Type 3, etc.
I would like to shuffle the order of the DataFrame's rows according to a list. \
For example, give a list [2, 4, 0, 3, 1, 5] and desired result should be:
    Col1  Col2  Col3  Type
2      7     8     9     2
4     13    14    15     3
0     1     2     3     1
3    10    11    12     2
1     4     5     6     1
5    16    17    18     3
...


How can I achieve this?


A:
<code>
import pandas as pd
import numpy as np


df = pd.DataFrame({'Col1': [1, 4, 7, 10, 13, 16],
                   'Col2': [2, 5, 8, 11, 14, 17],
                   'Col3': [3, 6, 9, 12, 15, 18],
                   'Type': [1, 1, 2, 2, 3, 3]})

### The answer which goes into the Context Code to run and check it works or not!

In [121]:
print(f"{samples['reference_code'][0]}")

def g(df, List):
    return df.iloc[List]

result = g(df.copy(), List)



In [122]:
print(f"{samples['metadata'][0]}")

{'problem_id': 0, 'library_problem_id': 0, 'library': 'Pandas', 'test_case_cnt': 1, 'perturbation_type': 'Origin', 'perturbation_origin_id': 0}


### The code context is the code to which the solution code gets attached and run to check our solution works or not

In [124]:
print(f"{samples['code_context'][0]}")

import pandas as pd
import numpy as np
import copy


def generate_test_case(test_case_id):
    def generate_ans(data):
        data = data
        df, List = data
        return df.iloc[List]

    def define_test_input(test_case_id):
        if test_case_id == 1:
            df = pd.DataFrame(
                {
                    "Col1": [1, 4, 7, 10, 13, 16],
                    "Col2": [2, 5, 8, 11, 14, 17],
                    "Col3": [3, 6, 9, 12, 15, 18],
                    "Type": [1, 1, 2, 2, 3, 3],
                }
            )
            List = np.random.permutation(len(df))
        return df, List

    test_input = define_test_input(test_case_id)
    expected_result = generate_ans(copy.deepcopy(test_input))
    return test_input, expected_result


def exec_test(result, ans):
    try:
        pd.testing.assert_frame_equal(result, ans, check_dtype=False)
        return 1
    except:
        return 0


exec_context = r"""
import pandas as pd
import numpy as np
df, List = te

In [123]:
print(f"{samples['prompt'][0]}")

Problem:
I have the following DataFrame:
    Col1  Col2  Col3  Type
0      1     2     3     1
1      4     5     6     1
2      7     8     9     2
3    10    11    12     2
4    13    14    15     3
5    16    17    18     3


The DataFrame is read from a CSV file. All rows which have Type 1 are on top, followed by the rows with Type 2, followed by the rows with Type 3, etc.
I would like to shuffle the order of the DataFrame's rows according to a list. \
For example, give a list [2, 4, 0, 3, 1, 5] and desired result should be:
    Col1  Col2  Col3  Type
2      7     8     9     2
4     13    14    15     3
0     1     2     3     1
3    10    11    12     2
1     4     5     6     1
5    16    17    18     3
...


How can I achieve this?


A:
<code>
import pandas as pd
import numpy as np


df = pd.DataFrame({'Col1': [1, 4, 7, 10, 13, 16],
                   'Col2': [2, 5, 8, 11, 14, 17],
                   'Col3': [3, 6, 9, 12, 15, 18],
                   'Type': [1, 1, 2, 2, 3, 3]})

## Helper Functions to execute our tasks

In [125]:
def extract_from_tags(text: str, start_tag: str, end_tag: str) -> str:
    start_index = len(start_tag) if text.startswith(start_tag) else 0
    end_index = text.find(end_tag, len(start_tag))
    end_index = end_index if end_index != -1 else len(text)
    return text[start_index:end_index]

def postprocess(code: str) -> str:
    
    code = extract_from_tags(code, "```python\n", "\n```")
    code = extract_from_tags(code, "```\n", "\n```")  # new pattern
    code = extract_from_tags(code, "<code>", "</code>")
    code = extract_from_tags(code, "", "</code>")
    code = extract_from_tags(code, "", "\nEND SOLUTION")
    code = extract_from_tags(code, "", "\n### END SOLUTION")
    return code.strip()


In [126]:
def get_solution(cmbagent_results):
    chat_history = cmbagent_results['chat_history']
    final_context = cmbagent_results['final_context']

    # Load target file path from metadata and read target answer
    #target_file = metadata['target_file_path']
    # target_file = "targets/problem_{index}.txt"
    # try:
    #     with open(target_file, 'r') as f:
    #         target_answer = f.read().strip()
    # except FileNotFoundError as e:
    #     return ["FAILED", f"Target file not found: {e}"]

    # Build path to result.txt
    result_path = os.path.join(
        final_context['work_dir'],
        final_context['database_path'],
        "result.txt"
    )
    result_path = os.path.abspath(result_path)

    # Read result answer
    try:
        with open(result_path, 'r') as f:
            result_answer = f.read().strip()
            return result_answer
    except FileNotFoundError as e:
        return f"Result file not found: {e}"

In [131]:
def my_agent(task, metadata):

    results = cmbagent.one_shot(task,
                                max_rounds=200,
                                #initial_agent=metadata['initial_agent'],
                                engineer_model='gpt-4o-mini'
                                )


    return get_solution(results)
    #return results

In [147]:
results = []

for i, sample in enumerate(samples):
    print(f"### Sample {i+1}: {sample['metadata']['problem_id']} ###")
    print(f"Write a short code following the given format and indentation. "
    f"Place the executable code between <code> and </code> tags, without any other non-executable things \n"
    f"Also save the code you place between <code> and </code> tags along with the tags in a result.txt file "
    f"Only provide the code completion needed. Don't repeat the context code."
    f"Prompt:\n {sample['prompt']}")

    # Run agent
    raw_output = my_agent( f"Write a short code following the given format and indentation. "
    f"Place the executable code between <code> and </code> tags, without any other non-executable things \n"
    f"Also save the code you place between <code> and </code> tags along with the tags in a result.txt file \n"
    f"Only provide the code completion needed. Don't repeat the context code. \n"
    f"DO not specifically code for the example given but for a more general purpose. \n"
    f"Also just complete the code written after A : and for checking your code runs or not run the code after A : after completing it \n"
    f"Also just complete the code written after A : and save it in results.txt. \n"
    f"Prompt:\n {sample['prompt']}\n"
    f"Finally save the code you place between <code> and </code> tags along with the tags in a \"result.txt\" file "
    , sample["metadata"]
    )
    #print(raw_output)  # Check structur
    
    solution_code = postprocess(raw_output)
    #solution_code = raw_output
    
    print("\nAgent Solution:\n", solution_code)

    # Build the full test script
    code_context = sample["code_context"]
    full_code = (
        f"{code_context}\n"
        + f"solution = '''{solution_code}'''\n"
        + "test_execution(solution)\n"
    )
    if "test_string(" in code_context:
        full_code += "test_string(solution)\n"

    print("\n--- Running Test ---")
    try:
        exec_locals = {}
        #exec(full_code, {}, exec_locals)
        exec(full_code)
        print("✅ Passed")
        results.append(("C", sample["metadata"]["problem_id"]))
    except Exception as e:
        print("❌ Failed")
        traceback.print_exc()
        results.append(("I", sample["metadata"]["problem_id"]))
    
    print("="*80)


### Sample 1: 0 ###
Write a short code following the given format and indentation. Place the executable code between <code> and </code> tags, without any other non-executable things 
Also save the code you place between <code> and </code> tags along with the tags in a result.txt file Only provide the code completion needed. Don't repeat the context code.Prompt:
 Problem:
I have the following DataFrame:
    Col1  Col2  Col3  Type
0      1     2     3     1
1      4     5     6     1
2      7     8     9     2
3    10    11    12     2
4    13    14    15     3
5    16    17    18     3


The DataFrame is read from a CSV file. All rows which have Type 1 are on top, followed by the rows with Type 2, followed by the rows with Type 3, etc.
I would like to shuffle the order of the DataFrame's rows according to a list. \
For example, give a list [2, 4, 0, 3, 1, 5] and desired result should be:
    Col1  Col2  Col3  Type
2      7     8     9     2
4     13    14    15     3
0     1     2     3

/opt/anaconda3/envs/cmbagentmain2/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:129: UserWarning: Update function string contains no variables. This is probably unintended.
  warnings.warn("Update function string contains no variables. This is probably unintended.")


_User (to chat_manager):

Write a short code following the given format and indentation. Place the executable code between <code> and </code> tags, without any other non-executable things 
Also save the code you place between <code> and </code> tags along with the tags in a result.txt file 
Only provide the code completion needed. Don't repeat the context code. 
DO not specifically code for the example given but for a more general purpose. 
Also just complete the code written after A : and for checking your code runs or not run the code after A : after completing it 
Also just complete the code written after A : and save it in results.txt. 
Prompt:
 Problem:
I have the following DataFrame:
    Col1  Col2  Col3  Type
0      1     2     3     1
1      4     5     6     1
2      7     8     9     2
3    10    11    12     2
4    13    14    15     3
5    16    17    18     3


The DataFrame is read from a CSV file. All rows which have Type 1 are on top, followed by the rows with Type 2, f

Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
gpt-4o-mini-2024-07-18,engineer,0.00051,3071,76,3147


engineer (to chat_manager):

**Code Explanation:**

The code creates a DataFrame and shuffles its rows based on a specified list of indices. The result is stored in the variable `result`.

**Modifications:** (Optional)

No modifications were necessary as the task was to complete the code.

**Python Code:**

<code>
result = df.iloc[List].reset_index(drop=True)
</code>

--------------------------------------------------------------------------------

Next speaker: engineer_response_formatter


>>>>>>>> USING AUTO REPLY...


...

Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
gpt-4o-2024-11-20,executor_response_formatter,0.01929,7490,56,7546


executor_response_formatter (to chat_manager):

***** Suggested tool call (call_1EgDDlV3KJjKQFIOHgOWC4U3): post_execution_transfer *****
Arguments: 
{"next_agent_suggestion":"engineer","execution_status":"failure"}
****************************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Group_Tool_Executor


>>>>>>>> EXECUTING FUNCTION post_execution_transfer...
Call ID: call_1EgDDlV3KJjKQFIOHgOWC4U3
Input arguments: {'next_agent_suggestion': 'engineer', 'execution_status': 'failure'}
_Group_Tool_Executor (to chat_manager):

***** Response from calling tool (call_1EgDDlV3KJjKQFIOHgOWC4U3) *****
Max number of code execution attempts (3) reached. Exiting.
**********************************************************************

--------------------------------------------------------------------------------

Next speaker: terminator


>>>>>>>> USING AUTO REPLY...


Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
gpt-4o-2024-11-20,terminator,0.00701,2800,1,2801


terminator (to chat_manager):

***** Suggested tool call (call_70pPZQVsmJDVYtKKfbR7XaHd): terminate_session *****
Arguments: 
{}
**********************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Group_Tool_Executor


>>>>>>>> EXECUTING FUNCTION terminate_session...
Call ID: call_70pPZQVsmJDVYtKKfbR7XaHd
Input arguments: {}
_Group_Tool_Executor (to chat_manager):

***** Response from calling tool (call_70pPZQVsmJDVYtKKfbR7XaHd) *****
Session terminated.
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (58479471-4bbc-46a2-a86f-bdb8742e2411): No next speaker selected

Displaying cost…

| Agent                       | Cost ($)    | Prompt Tokens | Completion Tokens | Total Tokens |
|:----------------------------|------------:|--------------:|-------

Traceback (most recent call last):
  File "/var/folders/f6/v55nk5k10bl5v1zpkfhq51h00000gn/T/ipykernel_46758/329129666.py", line 44, in <module>
    exec(full_code)
  File "<string>", line 54
    solution = '''Result file not found: [Errno 2] No such file or directory: '/Users/milind24/cmbagentmain/cmbagent/output/data/result.txt''''
                                                                                                                                              ^
SyntaxError: unterminated string literal (detected at line 54)


### Problem with CMBAgent is that it is neither giving the exact or similar code to run or saving the result.txt file even after mentioning explicitly twice that's why you will see it fails.

### Also the problem is CMBAgent is preparing a code specific for the example given but not in a general sense so it fails

### Putting the original solution into my code logic to check my code and logic works or not and it passes gracefully

In [150]:
for i, sample in enumerate(samples):
    # print(f"### Sample {i+1}: {sample['metadata']['problem_id']} ###")
    # print(f"Write a short code following the given format and indentation. "
    # f"Place the executable code between <code> and </code> tags, without any other non-executable things \n"
    # f"Also save the code you place between <code> and </code> tags along with the tags in a result.txt file "
    # f"Prompt:\n {sample['prompt']}")

    # # Run agent
    # raw_output = my_agent( f"Write a short code following the given format and indentation. "
    # f"Place the executable code between <code> and </code> tags, without any other non-executable things \n"
    # f"Also save the code you place between <code> and </code> tags along with the tags in a result.txt file \n"
    # f"Prompt:\n {sample['prompt']}\n"
    # f"Finally save the code you place between <code> and </code> tags along with the tags in a \"result.txt\" file "
    # , sample["metadata"]
    # )
    # #print(raw_output)  # Check structur
    
    # solution_code = postprocess(raw_output)
    #solution_code = raw_output
    solution_code = '''def g(df, List):
     return df.iloc[List]
result = g(df.copy(), List)'''



    print("\nAgent Solution:\n", solution_code)

    # Build the full test script
    code_context = sample["code_context"]
    full_code = (
        f"{code_context}\n"
        + f"solution = '''{solution_code}'''\n"
        + "test_execution(solution)\n"
    )
    if "test_string(" in code_context:
        full_code += "test_string(solution)\n"

    print("\n--- Running Test ---")
    try:
        exec_locals = {}
        #exec(full_code, {}, exec_locals)
        exec(full_code)
        print("✅ Passed")
        results.append(("C", sample["metadata"]["problem_id"]))
    except Exception as e:
        print("❌ Failed")
        traceback.print_exc()
        results.append(("I", sample["metadata"]["problem_id"]))
    
    print("="*80)


Agent Solution:
 def g(df, List):
     return df.iloc[List]
result = g(df.copy(), List)

--- Running Test ---
✅ Passed


In [151]:
print(f"{full_code}")

import pandas as pd
import numpy as np
import copy


def generate_test_case(test_case_id):
    def generate_ans(data):
        data = data
        df, List = data
        return df.iloc[List]

    def define_test_input(test_case_id):
        if test_case_id == 1:
            df = pd.DataFrame(
                {
                    "Col1": [1, 4, 7, 10, 13, 16],
                    "Col2": [2, 5, 8, 11, 14, 17],
                    "Col3": [3, 6, 9, 12, 15, 18],
                    "Type": [1, 1, 2, 2, 3, 3],
                }
            )
            List = np.random.permutation(len(df))
        return df, List

    test_input = define_test_input(test_case_id)
    expected_result = generate_ans(copy.deepcopy(test_input))
    return test_input, expected_result


def exec_test(result, ans):
    try:
        pd.testing.assert_frame_equal(result, ans, check_dtype=False)
        return 1
    except:
        return 0


exec_context = r"""
import pandas as pd
import numpy as np
df, List = te

In [152]:
exec(full_code)
print("✅ Passed")

✅ Passed


In [145]:
'''def g(df, List):
     return df.iloc[List]
result = g(df.copy(), List)'''

'def g(df, List):\n     return df.iloc[List]\nresult = g(df.copy(), List)'